In [0]:
%pip install databricks-sdk==0.39.0
dbutils.library.restartPython()

In [0]:
catalog='dev_bh_datascience'
dbName='ds_workshop'

In [0]:
spark.sql("use catalog dev_bh_datascience")
spark.sql("use database ds_workshop")

In [0]:
%sql 
-- To setup monitoring, load in the silver_transaction dataset
SELECT * from silver_transaction limit 10;

## Create monitor

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.catalog import MonitorTimeSeries
import os

In [0]:
# Define time windows to aggregate metrics over
GRANULARITIES = ["1 day"]                       

# Optionally define expressions to slice data with
SLICING_EXPRS = ["Category='Toys'"]  

In [0]:
# You must have `USE CATALOG` privileges on the catalog, and you must have `USE SCHEMA` privileges on the schema.
# If necessary, change the catalog and schema name here.
TABLE_NAME = f"{catalog}.{dbName}.silver_transaction"

# Define the timestamp column name
TIMESTAMP_COL = "TransactionDate"

# Enable Change Data Feed (CDF) to incrementally process changes to the table and make execution more efficient 
display(spark.sql(f"ALTER TABLE {TABLE_NAME} SET TBLPROPERTIES (delta.enableChangeDataFeed = true)"))

In [0]:
# Create a monitor using a Timeseries profile type. After the intial refresh completes, you can view the autogenerated dashboard from the Quality tab of the table in Catalog Explorer. 
print(f"Creating monitor for {TABLE_NAME}")

w = WorkspaceClient()

try:
  lhm_monitor = w.quality_monitors.create(
    table_name=TABLE_NAME, # Always use 3-level namespace
    time_series = MonitorTimeSeries(
      timestamp_col=TIMESTAMP_COL,
      granularities=GRANULARITIES
    ),
    assets_dir = os.getcwd(),
    output_schema_name=f"{catalog}.{dbName}"
  )
  
except Exception as lhm_exception:
  if "already exist" in str(lhm_exception):
    print(f"Monitor for {TABLE_NAME} already exists, retrieving monitor info:")
    lhm_monitor = w.quality_monitors.get(table_name=f"{TABLE_NAME}")

  else:
    raise lhm_exception

In [0]:
import time
from databricks.sdk.service.catalog import MonitorInfoStatus, MonitorRefreshInfoState

In [0]:
# Wait for monitor to be created
lhm_monitor = w.quality_monitors.get(table_name=f"{TABLE_NAME}")
while lhm_monitor.status == MonitorInfoStatus.MONITOR_STATUS_PENDING:
  lhm_monitor = w.quality_monitors.get(table_name=f"{TABLE_NAME}")
  time.sleep(10)

assert lhm_monitor.status == MonitorInfoStatus.MONITOR_STATUS_ACTIVE, "Error creating monitor"

refreshes = w.quality_monitors.list_refreshes(table_name=f"{TABLE_NAME}").refreshes
assert(len(refreshes) > 0)

run_info = refreshes[0]
while run_info.state in (MonitorRefreshInfoState.PENDING, MonitorRefreshInfoState.RUNNING):
  run_info = w.quality_monitors.get_refresh(table_name=f"{TABLE_NAME}", refresh_id=run_info.refresh_id)
  time.sleep(30)

assert run_info.state == MonitorRefreshInfoState.SUCCESS, "Monitor refresh failed"

## Orientation to the profile metrics table

The profile metrics table has the suffix _profile_metrics. For a list of statistics that are shown in the table, see the documentation (AWS|Azure).

For every column in the primary table, the profile table shows summary statistics for the baseline table and for the primary table. The column log_type shows INPUT to indicate statistics for the primary table, and BASELINE to indicate statistics for the baseline table. The column from the primary table is identified in the column column_name.
For TimeSeries type analysis, the granularity column shows the granularity corresponding to the row. For baseline table statistics, the granularity column shows null.
The table shows statistics for each value of each slice key in each time window, and for the table as whole. Statistics for the table as a whole are indicated by slice_key = slice_value = null.
In the primary table, the window column shows the time window corresponding to that row. For baseline table statistics, the window column shows null.
Some statistics are calculated based on the table as a whole, not on a single column. In the column column_name, these statistics are identified by :table.

In [0]:
# Display profile metrics table
profile_table = lhm_monitor.profile_metrics_table_name  
display(spark.sql(f"SELECT * FROM {profile_table}"))

In [0]:
profile_table

## Orientation to the drift metrics table

The drift metrics table has the suffix _drift_metrics. For a list of statistics that are shown in the table, see the documentation (AWS | Azure).

For every column in the primary table, the drift table shows a set of metrics that compare the current values in the table to the values at the time of the previous analysis run and to the baseline table. The column drift_type shows BASELINE to indicate drift relative to the baseline table, and CONSECUTIVE to indicate drift relative to a previous time window. As in the profile table, the column from the primary table is identified in the column column_name.
For TimeSeries type analysis, the granularity column shows the granularity corresponding to that row.
The table shows statistics for each value of each slice key in each time window, and for the table as whole. Statistics for the table as a whole are indicated by slice_key = slice_value = null.
The window column shows the the time window corresponding to that row. The window_cmp column shows the comparison window. If the comparison is to the baseline table, window_cmp is null.
Some statistics are calculated based on the table as a whole, not on a single column. In the column column_name, these statistics are identified by :table.

In [0]:
# Display the drift metrics table
drift_table = lhm_monitor.drift_metrics_table_name  
display(spark.sql(f"SELECT * FROM {drift_table}"))

In [0]:
drift_table

### One col

In [0]:
display(spark.sql(f"SELECT * FROM {profile_table} where column_name = 'TotalPurchaseAmount'"))

In [0]:
display(spark.sql(f"SELECT * FROM {drift_table} where column_name = 'TotalPurchaseAmount'"))

## View the Autogenerated Dashboard

After the intial refresh completes, you can view the autogenerated dashboard from the Quality tab of the silver_transactions table in Catalog Explorer. The dashboard visualizes metrics in the following sections:

Data Volume: Check if transaction volume is expected or if there's been changes with seasonality
Data Integrity: Identify the columns with a high % of nulls or zeros and view their distribution over time
Numerical Distribution Change: Identify numerical anomalies and view the Range of values over time
Categorical Distribution Change: Identify categorical anomalies like PreferredPaymentMethod and view the distribution of values time
Profiling: Explore the numerical and categorical data profile over time

## Delete tables

In [0]:
# Uncomment the following line of code to clean up the monitor (if you wish to run the quickstart on this table again).
w.quality_monitors.delete(TABLE_NAME)